# Tutorial on how to use mescal

*mescal* is a [Brightway](https://docs.brightway.dev/en/latest/)-powered Python package that helps you to integrate Life-Cycle Assessment (LCA) in your energy system model.

In [ ]:
# Import the required libraries, including mescal
from mescal import *
import pandas as pd

## Your data

For mescal to understand the structure of your energy system model, you need to provide it with a set of dataframes.

Mandatory dataframes are:
- A mapping between the energy technologies and resources of the energy system model, and Life-Cycle Inventories datasets (LCI) from an LCI database (e.g., ecoinvent). The mapping should be provided in a dataframe with the following columns:
    - Name: the name of the energy technology or resource in the energy model 
    - Type: the type of the energy technology or resource (i.e., 'construction', 'operation', or 'resource')  
    - Activity: the name of the activity of the energy technology or resource in the LCI database
    - Product: the name of the product of the energy technology or resource in the LCI database
    - Region: the name of the region of the energy technology or resource in the LCI database
    - Database: the name of the database in your brightway project
- A set of conversion factors, including the difference of units and assumptions (e.g., lifetime, efficiency, capacity factors, etc.) between the energy system model and the LCI database. The conversion factors should be provided in a dataframe with the following columns:
    - Name: the name of the energy technology or resource in the energy model 
    - Type: the type of the energy technology or resource (i.e., 'construction', 'operation', or 'resource')
    - CF: the numerical value of the conversion factor
- The input and output flows if energy technologies. It should be provided in a dataframe with the following columns:
    - Name: the name of the energy technology in the energy model
    - Flow: the name of the input or output flow
    - Value: the numerical value of the flow (negative if input, positive if output)   
- A mapping between the energy model flows and CPC categories. The mapping should be provided in a dataframe with the following columns:
    - Name: the name of the energy flow in the energy model
    - CPC: the list of names of corresponding CPC categories

Optional dataframes are:
- A set of composition of technologies, i.e., if one technology or resource in the energy model should be represented by a combination of LCI datasets. The composition should be provided in a dataframe with the following columns:
    - Name: the name of the main energy technology or resource in the energy model 
    - Composition: the list of names of subcomponents
- A set of technologies with specific requirements. For instance, this stands for energy technologies without a construction phase, mobility technologies (if mismatch fuel in the LCI dataset), bioprocesses (if mismatch fuel in the LCI dataset). The requirements should be provided in a dataframe with the following columns:
    - Name: the name of the energy technology in the energy model
    - Requirement: the list of requirements  